In [1]:
checkpoint_path = 'weights/Crop/Plain/Crop_res50_plain_082723-0-epoch=14-valid_mac_acc=91.38.ckpt'

import torch

# Load the checkpoint
checkpoint_path = 'weights/Crop/Plain/Crop_res50_plain_082723-0-epoch=14-valid_mac_acc=91.38.ckpt'
checkpoint = torch.load(checkpoint_path)

# Inspecting the contents
print(checkpoint.keys())  # List all keys in the checkpoint

# Example: Print model state_dict
if 'model_state_dict' in checkpoint:
    print(checkpoint['model_state_dict'])
    
# Example: Print other relevant metrics
if 'epoch' in checkpoint:
    print(f"Epoch: {checkpoint['epoch']}")
if 'valid_mac_acc' in checkpoint:
    print(f"Validation Accuracy: {checkpoint['valid_mac_acc']}")

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])
Epoch: 14


In [2]:
# Access the state_dict
state_dict = checkpoint['state_dict']

# Print the keys in the state_dict (which represent layer names)
print(state_dict.keys())

# Optionally, print the weights for a specific layer
layer_name = 'your_layer_name'  # Replace with your specific layer name
if layer_name in state_dict:
    print(state_dict[layer_name])


odict_keys(['net.feature.conv1.weight', 'net.feature.bn1.weight', 'net.feature.bn1.bias', 'net.feature.bn1.running_mean', 'net.feature.bn1.running_var', 'net.feature.bn1.num_batches_tracked', 'net.feature.layer1.0.conv1.weight', 'net.feature.layer1.0.bn1.weight', 'net.feature.layer1.0.bn1.bias', 'net.feature.layer1.0.bn1.running_mean', 'net.feature.layer1.0.bn1.running_var', 'net.feature.layer1.0.bn1.num_batches_tracked', 'net.feature.layer1.0.conv2.weight', 'net.feature.layer1.0.bn2.weight', 'net.feature.layer1.0.bn2.bias', 'net.feature.layer1.0.bn2.running_mean', 'net.feature.layer1.0.bn2.running_var', 'net.feature.layer1.0.bn2.num_batches_tracked', 'net.feature.layer1.0.conv3.weight', 'net.feature.layer1.0.bn3.weight', 'net.feature.layer1.0.bn3.bias', 'net.feature.layer1.0.bn3.running_mean', 'net.feature.layer1.0.bn3.running_var', 'net.feature.layer1.0.bn3.num_batches_tracked', 'net.feature.layer1.0.downsample.0.weight', 'net.feature.layer1.0.downsample.1.weight', 'net.feature.layer

In [ ]:
# Access optimizer states
optimizer_states = checkpoint['optimizer_states']

# Print the optimizer states
for i, state in enumerate(optimizer_states):
    print(f"Optimizer {i}: {state}")


In [3]:
# Access hyper parameters
hyper_parameters = checkpoint['hyper_parameters']
print(hyper_parameters)


{'conf_id': 'Crop_res50_plain_082723', 'algorithm': 'Plain', 'log_dir': 'Crop', 'num_epochs': 15, 'log_interval': 10, 'parallel': 0, 'dataset_root': './data/imgs', 'dataset_name': 'Custom_Crop', 'annotation_dir': './data/imgs', 'split_path': './data/imgs/tiger_binary.csv', 'test_size': 0.2, 'val_size': 0.2, 'split_data': True, 'split_type': 'random', 'batch_size': 32, 'num_workers': 8, 'num_classes': 2, 'model_name': 'PlainResNetClassifier', 'num_layers': 50, 'weights_init': 'ImageNet', 'lr_feature': 0.01, 'momentum_feature': 0.9, 'weight_decay_feature': 0.0005, 'lr_classifier': 0.01, 'momentum_classifier': 0.9, 'weight_decay_classifier': 0.0005, 'step_size': 10, 'gamma': 0.1, 'evaluate': None, 'val': False, 'predict': False, 'predict_root': '', 'id_to_labels': {0: 'Not Tiger', 1: 'Tiger'}}


In [19]:
import torch
import torch.nn as nn
from torchvision.models.resnet import Bottleneck, ResNet

class ResNetBackbone(ResNet):
    def __init__(self, block, layers, **kwargs):
        super(ResNetBackbone, self).__init__(block=block, layers=layers, **kwargs)

    def _forward_impl(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return x

class PlainResNetClassifier(nn.Module):
    def __init__(self, num_cls=2, num_layers=50):
        super(PlainResNetClassifier, self).__init__()
        self.num_cls = num_cls
        self.num_layers = num_layers

        # Initialize the network
        self.setup_net()

    def setup_net(self):
        block = Bottleneck
        layers = [3, 4, 6, 3]

        # Constructing the feature extractor and classifier
        self.feature = ResNetBackbone(block, layers)
        self.classifier = nn.Linear(512 * block.expansion, self.num_cls)
    
    def forward(self, x):
        x = self.feature(x)  # Pass through the feature extractor
        x = self.classifier(x)  # Pass through the classifier
        return x

In [ ]:
!pip install torchsummary

In [5]:
from torchsummary import summary

# Instantiate the model without pre-trained weights
model = PlainResNetClassifier(num_cls=2, num_layers=50)

# Print the model architecture summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
summary(model, input_size=(3, 224, 224))  # Change the input size if necessary

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [6]:
# Load weights directly from the checkpoint
checkpoint_path = 'weights/Crop/Plain/Crop_res50_plain_082723-0-epoch=14-valid_mac_acc=91.38.ckpt'
checkpoint = torch.load(checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['state_dict'], strict=False)

# Print the model architecture summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
summary(model, input_size=(3, 224, 224))  # Change the input size if necessary

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,